<a href="https://colab.research.google.com/github/Brahmee-Rout/AI-CLUB-TASK-SUBMISSION/blob/main/voice_emotion_recognition_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install numpy librosa matplotlib scikit-learn tensorflow soundfile
import librosa
import numpy as np

SAMPLE_RATE = 22050
DURATION = 3
N_MELS = 128
MAX_LEN = 128

def extract_log_mel(file_path):
    y, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)

    # Silence trimming
    y, _ = librosa.effects.trim(y)

    mel = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_mels=N_MELS
    )

    log_mel = librosa.power_to_db(mel)

    # Padding
    if log_mel.shape[1] < MAX_LEN:
        pad_width = MAX_LEN - log_mel.shape[1]
        log_mel = np.pad(log_mel, pad_width=((0,0),(0,pad_width)))
    else:
        log_mel = log_mel[:, :MAX_LEN]

    return log_mel
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}
import os

X, y = [], []
import kagglehub


path = kagglehub.dataset_download("orvile/ravdess-dataset")

print("Path to dataset files:", path)
dataset_path = path

for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".wav"):
            emotion_code = file.split("-")[2]
            feature = extract_log_mel(os.path.join(root, file))
            X.append(feature)
            y.append(emotion_code)

X = np.array(X)
y = np.array(y)
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)
from sklearn.model_selection import train_test_split

X = X[..., np.newaxis]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_cat, test_size=0.2, stratify=y_cat, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=X_train.shape[1:]),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    GlobalAveragePooling2D(),

    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(8, activation='softmax')
])

model.compile(
    optimizer=Adam(0.0003),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32
)
model.save("model.h5")
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, y_pred_labels, target_names=le.classes_))
print(confusion_matrix(y_true, y_pred_labels))
import librosa
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model("model.h5")

def predict_emotion(file_path):
    mel = extract_log_mel(file_path)
    mel = mel[np.newaxis, ..., np.newaxis]

    preds = model.predict(mel)[0]
    emotion = le.inverse_transform([np.argmax(preds)])[0]
    confidence = np.max(preds) * 100

    print(f"Emotion: {emotion}")
    print(f"Confidence: {confidence:.2f}%")

# Example usage
predict_emotion("test.wav")


Using Colab cache for faster access to the 'ravdess-dataset' dataset.
Path to dataset files: /kaggle/input/ravdess-dataset


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,112 (434.03 KB)

...

Epoch 1/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 64s 986ms/step - accuracy: 0.2463 - loss: 1.9243 - val_accuracy: 0.1755 - val_loss: 3.3760
Epoch 2/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 59s 953ms/step - accuracy: 0.3410 - loss: 1.6527 - val_accuracy: 0.2653 - val_loss: 1.8651
Epoch 3/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.4020 - loss: 1.5535 - val_accuracy: 0.3510 - val_loss: 1.6234
Epoch 4/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.3867 - loss: 1.5511 - val_accuracy: 0.4204 - val_loss: 1.5022
Epoch 5/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 51s 821ms/step - accuracy: 0.4063 - loss: 1.4937 - val_accuracy: 0.4041 - val_loss: 1.5118
Epoch 6/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 53s 852ms/step - accuracy: 0.4457 - loss: 1.4363 - val_accuracy: 0.4367 - val_loss: 1.4209
Epoch 7/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 59s 960ms/step - accuracy: 0.4374 - loss: 1.4454 - val_accuracy: 0.4653 - val_loss: 1.3518
Epoch 8/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 79s 909ms/step - accuracy: 0.4526 - loss: 1.3793 - val_accuracy: 

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step


              precision    recall  f1-score   support

          01       0.83      0.23      0.36        22
          02       0.95      0.46      0.62        39
          03       0.75      0.17      0.28        35
          04       0.52      0.32      0.39        38
          05       0.93      0.48      0.64        29
          06       0.40      0.89      0.55        46
          07       0.35      1.00      0.51        18
          08       0.67      0.74      0.70        19

    accuracy                           0.52       246
   macro avg       0.68      0.54      0.51       246
weighted avg       0.67      0.52      0.50       246

[[ 5  0  2  6  0  3  4  2]
 [ 0 18  0  5  0  7  9  0]
 [ 0  0  6  0  0 25  2  2]
 [ 1  1  0 12  0 14 10  0]
 [ 0  0  0  0 14 10  3  2]
 [ 0  0  0  0  1 41  3  1]
 [ 0  0  0  0  0  0 18  0]
 [ 0  0  0  0  0  2  3 14]]


/tmp/ipython-input-3986051693.py:11: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'test.wav'

In [4]:
import sys
import numpy as np
import librosa
import tensorflow as tf

# Load model
MODEL_PATH = "model.h5"
model = tf.keras.models.load_model(MODEL_PATH)

# Emotion labels (RAVDESS order)
emotion_labels = [
    "neutral", "calm", "happy", "sad",
    "angry", "fearful", "disgust", "surprised"
]

SAMPLE_RATE = 22050
DURATION = 3
N_MELS = 128
MAX_LEN = 128

def extract_log_mel(file_path):
    y, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
    y, _ = librosa.effects.trim(y)

    mel = librosa.feature.melspectrogram(
        y=y, sr=sr, n_mels=N_MELS
    )
    log_mel = librosa.power_to_db(mel)

    if log_mel.shape[1] < MAX_LEN:
        pad = MAX_LEN - log_mel.shape[1]
        log_mel = np.pad(log_mel, ((0,0),(0,pad)))
    else:
        log_mel = log_mel[:, :MAX_LEN]

    return log_mel

def predict_emotion(audio_path):
    mel = extract_log_mel(audio_path)
    mel = mel[np.newaxis, ..., np.newaxis]

    preds = model.predict(mel)[0]
    idx = np.argmax(preds)

    print(f"Predicted Emotion : {emotion_labels[idx]}")
    print(f"Confidence        : {preds[idx]*100:.2f}%")

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: python predict.py <audio.wav>")
        sys.exit(1)

    predict_emotion(sys.argv[1])


Usage: python predict.py <audio.wav>


SystemExit: 1

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
